In [2]:
%pip install openai -U --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
from openai import AzureOpenAI
import time
import json
import os
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    api_version="2024-02-15-preview",
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [4]:
# Create an assistant
assistant = client.beta.assistants.create(
    name="Math Assist",
    instructions="You are an AI assistant that can write code to help answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4" #You must replace this value with the deployment name for your model.
)

# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status

# Wait till the assistant has responded
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    status = run.status

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_JVll8GQ1SyerRL5ZrgpDpqsD",
      "assistant_id": "asst_KyEGjGRx3Eg800EMlic078Kx",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\)."
          },
          "type": "text"
        }
      ],
      "created_at": 1707273798,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_VddilsoKqhqSE2JOapPiTkjh",
      "thread_id": "thread_Tk6cdzJrLtb93rHwjPhp2bem"
    },
    {
      "id": "msg_XPIDjHia8Ww03NakMAaU8kFC",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "I need to solve the equation `3x + 11 = 14`. Can you help me?"
          },
          "type": "text"
        }
      ],
      "created_at": 1707273792,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
  